In [1]:
import pandas as pd
import numpy as np
import math
import folium
from pandasql import sqldf

In [3]:
dados = pd.read_csv("vivareal-boaviagem.csv")

In [3]:
dados.head()

,título,preço,condomínio,iptu,área,quartos,banheiros,vagas,url,latitude,longitude
0,"Apartamento na Rua dos Navegantes, 1033, Boa V...",R$ 659.000,R$ 1.200,R$ 3.700,151.0,3.0,2.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.123251,-34.897370
1,"Apartamento na Rua Ministro Nelsom Hungria, 15...",R$ 270.000,R$ 744,NaN,91.0,3.0,3.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.110229,-34.897039
2,Apartamento na Rua Professor Augusto Lins e Si...,R$ 510.000,R$ 450,R$ 1.200,70.0,2.0,2.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.144108,-34.907494
3,"Apartamento na Rua Doutor Pedro de Melo Cahú, ...",R$ 1.250.000,R$ 800,NaN,163.0,4.0,5.0,3.0,https://www.vivareal.com.br/imovel/apartamento...,-8.110377,-34.895452
4,"Apartamento na Rua Luiz Pimentel, 77, Boa Viag...",R$ 350.000,R$ 550,R$ 2.000,65.0,2.0,1.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.137637,-34.905575


In [5]:
CENTRO = (-8.1179855, -34.894511)
mapa = folium.Map(CENTRO, zoom_start=14)
for i, linha in dados[:500].iterrows():
    if linha['latitude'] and linha['longitude']:
        folium.Marker(
            (linha['latitude'], linha['longitude']), popup=linha['título']
        ).add_to(mapa)
display(mapa)

In [6]:
dados.describe()

,área,quartos,banheiros,vagas,latitude,longitude
count,2767.000000,2769.000000,2740.000000,2745.000000,2770.000000,2770.000000
mean,114.908927,3.087396,2.749270,1.842259,-8.121809,-34.900822
std,64.767678,0.735861,1.276454,0.806474,0.012376,0.005582
min,25.000000,1.000000,1.000000,1.000000,-8.161280,-34.915389
25%,82.000000,3.000000,2.000000,1.000000,-8.133590,-34.905453
50%,100.000000,3.000000,3.000000,2.000000,-8.117113,-34.901115
75%,136.000000,4.000000,3.000000,2.000000,-8.111765,-34.897086
max,820.000000,6.000000,7.000000,20.000000,-8.051061,-34.877393


In [30]:
def limpar_precos(valor):
    try:
        valor = float(valor.replace("R$", "").replace(".", "").replace(",", ".").strip())
    except AttributeError:
        valor = np.nan
    return valor

In [31]:
dados['preço'] = dados['preço'].apply(limpar_precos)
dados['condomínio'] = dados['condomínio'].apply(limpar_precos)
dados['iptu'] = dados['iptu'].apply(limpar_precos)

In [38]:
math.isnan(dados['condomínio'][6])

True

In [7]:
len(dados)

2770

In [10]:
dados[dados['preço'].isnull() == True]

,título,preço,condomínio,iptu,área,quartos,banheiros,vagas,url,latitude,longitude


In [23]:
dados = dados[dados['condomínio'].isnull() == False]

In [24]:
len(dados)

1248

In [25]:
len(dados[dados['iptu'].isnull() == True])

453

In [26]:
sql = 'select * from imoveis where imoveis.quartos in (2, 3)'
dados = sqldf(sql, {'imoveis': dados})

In [27]:
len(dados)

855

In [28]:
dados.describe()

,área,quartos,banheiros,vagas,latitude,longitude
count,855.000000,855.000000,851.000000,845.000000,855.000000,855.000000
mean,97.408187,2.777778,2.515864,1.616568,-8.123197,-34.901673
std,36.693690,0.415983,1.104289,0.569535,0.011633,0.005779
min,35.000000,2.000000,1.000000,1.000000,-8.147307,-34.915389
25%,75.000000,3.000000,2.000000,1.000000,-8.133410,-34.906492
50%,96.000000,3.000000,2.000000,2.000000,-8.120260,-34.901388
75%,108.000000,3.000000,3.000000,2.000000,-8.114926,-34.897818
max,406.000000,3.000000,6.000000,4.000000,-8.087054,-34.882707


In [29]:
dados.head(10)

,título,preço,condomínio,iptu,área,quartos,banheiros,vagas,url,latitude,longitude
0,"Apartamento na Rua dos Navegantes, 1033, Boa V...",659000,1200,3700,151.0,3.0,2.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.123251,-34.897370
1,"Apartamento na Rua Ministro Nelsom Hungria, 15...",270000,744,None,91.0,3.0,3.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.110229,-34.897039
2,Apartamento na Rua Professor Augusto Lins e Si...,510000,450,1200,70.0,2.0,2.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.144108,-34.907494
3,"Apartamento na Rua Luiz Pimentel, 77, Boa Viag...",350000,550,2000,65.0,2.0,1.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.137637,-34.905575
4,"Apartamento na Rua João Cardoso Aíres, 305, Bo...",570000,900,None,110.0,3.0,4.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.142826,-34.907311
5,"Apartamento na Rua Sansão Ribeiro, 120, Boa Vi...",350000,580,None,85.0,3.0,2.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.130643,-34.910058
6,"Apartamento na Rua Visconde de Jequitinhonha, ...",550000,700,None,110.0,3.0,1.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.142542,-34.906165
7,Apartamento na Rua Professor Augusto Lins e Si...,500000,1130,None,136.0,3.0,1.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.141113,-34.906314
8,"Apartamento na Rua Antônio Passos, Boa Viagem ...",595000,650,None,88.0,3.0,3.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.135343,-34.904837
9,Apartamento na Avenida Fernando Simões Barbosa...,370000,500,None,82.0,2.0,2.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.120180,-34.901595


In [32]:
dados_completos = dados[dados['iptu'].isnull() == False]

In [10]:
len(dados_completos)

800

In [11]:
dados_completos.head()

,título,preço,condomínio,iptu,área,quartos,banheiros,vagas,url,latitude,longitude
0,"Apartamento na Rua dos Navegantes, 1033, Boa V...",R$ 659.000,R$ 1.200,R$ 3.700,151.0,3.0,2.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.123251,-34.897370
2,Apartamento na Rua Professor Augusto Lins e Si...,R$ 510.000,R$ 450,R$ 1.200,70.0,2.0,2.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.144108,-34.907494
4,"Apartamento na Rua Luiz Pimentel, 77, Boa Viag...",R$ 350.000,R$ 550,R$ 2.000,65.0,2.0,1.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.137637,-34.905575
11,"Apartamento 167 m² em Boa Viagem em Recife, po...",R$ 1.100.000,R$ 900,R$ 3.200,167.0,4.0,4.0,2.0,https://www.vivareal.com.br/imovel/apartamento...,-8.110993,-34.891481
15,Apartamento na Rua Professor Augusto Lins e Si...,R$ 370.000,R$ 500,R$ 120,65.0,2.0,3.0,1.0,https://www.vivareal.com.br/imovel/apartamento...,-8.143386,-34.907124


In [33]:
media_iptu = dados_completos['iptu'].mean()
media_area = dados_completos['área'].mean()

In [45]:
def calcula_iptu(linha):
    if linha['iptu'] is np.nan:
        return ((media_iptu / media_area) * linha['área'])
    return linha['iptu']

In [46]:
dados['iptu'] = dados.apply(calcula_iptu, axis=1)

In [36]:
media_iptu

1142.51875

In [26]:
dados_completos['área'].mean()

119.24375

In [27]:
dados_completos['iptu'].describe()

count     800
unique    221
top       400
freq      101
Name: iptu, dtype: object

In [39]:
dados["iptu"].isnull().values.any()

True

In [55]:
pd.isnull(dados.iloc[1]["iptu"])

True

In [56]:
dados.to_csv?